In [1]:
# pip install -r ../requirements.txt

In [1]:
from utils import utils
from app import run_app, App
from core.gen_topics import gen_topics
from graphneo4j import GraphNeo4j
import warnings
warnings.filterwarnings("ignore") # supress all the warnigns

settings = utils.get_settings_notebook('../settings.json')
utils.pretty_print(settings)


/opt/Anaconda/anaconda3/envs/ucsd1/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/Anaconda/anaconda3/envs/ucsd1/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


{
    "crawler": {
        "on": true
    },
    "extract": {
        "awards_nominated": {
            "filepath_or_buffer": "files/archive/award_nominated.csv",
            "sep": ","
        },
        "awards_won": {
            "filepath_or_buffer": "files/archive/award_won.csv",
            "sep": ","
        },
        "cast": {
            "filepath_or_buffer": "files/archive/AllMoviesCastingRaw.csv",
            "sep": ";"
        },
        "directors": {
            "filepath_or_buffer": "files/archive/900_acclaimed_directors_awards.csv",
            "sep": ";"
        },
        "movies": {
            "filepath_or_buffer": "files/archive/AllMoviesDetailsCleaned.csv",
            "sep": ";"
        },
        "topics": {
            "filepath_or_buffer": "files/archive/topic_words.csv",
            "sep": ","
        },
        "topics_movies": {
            "filepath_or_buffer": "files/archive/movie_topic_ids.csv",
            "sep": ","
        }
    },
    "load": {
    

## Load the data from csv files

In [2]:
app = App(settings)
dfs = app.extract_transform()

MATCH (n) DETACH DELETE n
=========  Crwaler award_won ========
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 598 entries, 0 to 597
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   award_company  598 non-null    object
 1   award_type     598 non-null    object
 2   movie          597 non-null    object
 3   person         597 non-null    object
 4   year           598 non-null    object
dtypes: object(5)
memory usage: 23.5+ KB
None
========= end ========
=========  Crwaler award_nominated ========
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2832 entries, 0 to 2831
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   award_company  2832 non-null   object
 1   award_type     2832 non-null   object
 2   movie          2816 non-null   object
 3   person         2243 non-null   object
 4   year           2832 non-null   object
dtypes: obj

## Generate topics
  - Use this section to generate the csv's for topics and keywords

In [3]:
movie_topics = gen_topics()
# (df_movie_topics, df_topic_words) = movie_topics.batch_process(data_df['movies'][data_df['movies']['lang'] == 'en'])
(df_movie_topics, df_topic_words) = movie_topics.batch_process(dfs['movies'][
    dfs['movies']['overview'].apply(lambda x: x is not None and len(x) > 30)
])
df_topic_words.reset_index(drop=True, inplace=True)
df_movie_topics.reset_index(drop=True, inplace=True)
display(df_topic_words)
display(df_movie_topics)
df_topic_words.to_csv('./files/archive/topic_words.csv', index=True)
df_movie_topics.to_csv('./files/archive/movie_topic_ids.csv', index=True)

2022-12-10 09:18:37 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2022-12-10 09:18:37,186 : INFO : Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2022-12-10 09:18:38 INFO: Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| depparse     | combined  |
| sentiment    | sstplus   |
| constituency | wsj       |
| ner          | ontonotes |

2022-12-10 09:18:38,534 : INFO : Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
|

,0,1,2,3,4,5,6,7,8,9
0,"""bill""","""life""","""make""","""last""","""list""","""live""","""death""","""wrong""","""payback""","""shoot"""
1,"""film""","""suicide""","""life""","""escape""","""force""","""evidence""","""hold""","""master""","""star""","""political"""
2,"""life""","""mission""","""exist""","""record""","""set""","""daughter""","""help""","""man""","""film""","""green"""
3,"""put""","""send""","""get""","""force""","""back""","""unit""","""want""","""boot""","""cheat""","""kill"""
4,"""man""","""young""","""commit""","""start""","""death""","""involve""","""garbage""","""much""","""worker""","""nikander"""
...,...,...,...,...,...,...,...,...,...,...
3995,"""allow""","""city""","""station""","""lucchini""","""conne""","""creation""","""include""","""involve""","""mathematician""","""alain"""
3996,"""space""","""base""","""film""","""anime""","""lot""","""woman""","""home""","""remake""","""adventurer""","""monster"""
3997,"""queen""","""castle""","""daughter""","""language""","""royalty""","""direct""","""version""","""german""","""make""","""marry"""
3998,"""girl""","""life""","""filmmaker""","""meet""","""follow""","""break""","""courage""","""story""","""bleak""","""find"""


,movie_id,topic
0,2,4
1,3,4
2,12,7
3,13,8
4,14,2
...,...,...
11995,461955,3998
11996,462894,3992
11997,463019,3990
11998,463026,3990


## Load the dataframes into graph database (Neo4j)

In [5]:
app.etl()

=========  Crwaler award_won ========
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 598 entries, 0 to 597
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   award_company  598 non-null    object
 1   award_type     598 non-null    object
 2   movie          597 non-null    object
 3   person         597 non-null    object
 4   year           598 non-null    object
dtypes: object(5)
memory usage: 23.5+ KB
None
========= end ========
=========  Crwaler award_nominated ========
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2832 entries, 0 to 2831
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   award_company  2832 non-null   object
 1   award_type     2832 non-null   object
 2   movie          2816 non-null   object
 3   person         2243 non-null   object
 4   year           2832 non-null   object
dtypes: object(5)
memory usage: 110.8

## Display the final dataframe for the movies

In [6]:
dfs['movies']

,movie_id,imdb_id,lang,title,overview,vote_average,vote_count,budget,revenue,runtime,genres,actors,directors,companies,awards_won,awards_nominated,topics
0,2,tt0094675,fi,Ariel,Taisto Kasurinen is a Finnish coal miner whose...,7.1,40,0,0,69.0,"[{'name': 'Drama'}, {'name': 'Crime'}]","[{'name': 'Matti Pellonpaa', 'gender': 2}, {'n...","[{'name': 'Aki Kaurismaki', 'gender': 0.0}]",[{'name': 'Villealfa Filmproduction Oy'}],[],[],[{'topic_id': 4}]
1,3,tt0092149,fi,Shadows in Paradise,"An episode in the life of Nikander, a garbage ...",7.0,32,0,0,76.0,"[{'name': 'Drama'}, {'name': 'Comedy'}]","[{'name': 'Matti Pellonpaa', 'gender': 2}, {'n...","[{'name': 'Aki Kaurismaki', 'gender': 0.0}]",[{'name': 'Villealfa Filmproduction Oy'}],[],[],[{'topic_id': 4}]
2,12,tt0266543,en,Finding Nemo,"Nemo, an adventurous young clownfish, is unexp...",7.6,5531,94000000,940335536,100.0,"[{'name': 'Animation'}, {'name': 'Family'}]","[{'name': 'Albert Brooks', 'gender': 2}, {'nam...","[{'name': 'Andrew Stanton', 'gender': 2.0}]",[{'name': 'Pixar Animation Studios'}],[],[],[{'topic_id': 7}]
3,13,tt0109830,en,Forrest Gump,A man with a low IQ has accomplished great thi...,8.2,7204,55000000,677945399,142.0,"[{'name': 'Comedy'}, {'name': 'Drama'}, {'name...","[{'name': 'Tom Hanks', 'gender': 2}, {'name': ...","[{'name': 'Robert Zemeckis', 'gender': 2.0}]",[{'name': 'Paramount Pictures'}],"[{'award_id': 66}, {'award_id': 160}, {'award_...",[{'award_id': 1498}],[{'topic_id': 8}]
4,14,tt0169547,en,American Beauty,"Lester Burnham, a depressed suburban father in...",7.9,2994,15000000,356296601,122.0,[{'name': 'Drama'}],"[{'name': 'Kevin Spacey', 'gender': 2}, {'name...","[{'name': 'Sam Mendes', 'gender': 2.0}]",[{'name': 'DreamWorks SKG'}],"[{'award_id': 71}, {'award_id': 165}, {'award_...","[{'award_id': 259}, {'award_id': 1523}, {'awar...",[{'topic_id': 2}]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12149,468134,tt0174108,en,Paviljon VI,A doctor from provincial town in Tsardom of Ru...,0.0,0,0,0,97.0,[{'name': 'Drama'}],[],"[{'name': 'Lucian Pintilie', 'gender': 2.0}]",[{'name': 'Centar Film'}],[],[],[]
12150,468188,tt0020291,en,Prisoners,Prisoners was a 1929 American film directed by...,0.0,0,0,0,87.0,[{'name': 'Drama'}],"[{'name': 'Ian Keith', 'gender': 2}, {'name': ...","[{'name': 'William A Seiter', 'gender': 2.0}]",[{'name': 'First National Pictures'}],[],[],[]
12151,468335,tt0020128,en,Madonna of Avenue A,Madonna of Avenue A is a 1929 talking drama fi...,0.0,0,0,0,71.0,[{'name': 'Drama'}],"[{'name': 'Dolores Costello', 'gender': 1.0}, ...","[{'name': 'Michael Curtiz', 'gender': 0.0}]",[{'name': 'Warner Bros.'}],[],[],[]
12152,468749,tt0083115,en,Splendor in the Grass,Splendor in the Grass is a made-for-TV movie s...,0.0,0,0,0,100.0,[{'name': 'Drama'}],"[{'name': 'Michelle Pfeiffer', 'gender': 1.0},...","[{'name': 'Richard C Sarafian', 'gender': 0.0}]",[{'name': 'none'}],[],[],[]


## ADD KEYWORDS NODE

In [9]:
graph4j = GraphNeo4j(**{**settings['neo4j'], 'clear_data': False})

In [10]:
qry = '''
    MATCH (t:Topic)<-[]-(m:Movie) UNWIND(t.topics) as keyword
    MERGE (k:Keyword {keyword: keyword})
    MERGE (k)<-[:HAS_KEYWORD]-(m)
'''
graph4j.run_query(qry, {})

MATCH (t:Topic)<-[]-(m:Movie) UNWIND(t.topics) as keyword
    MERGE (k:Keyword {keyword: keyword})
    MERGE (k)<-[:HAS_KEYWORD]-(m)
